In [1]:
import os
os.chdir(os.path.join(os.path.abspath(os.path.expanduser("~")), "Documents", "Personal", "Test"))
from scripts.tools.issuetracker import IssuetrackerAPI as _ISAPI, Issue
import pickle

import datetime, dateutil
from_date = datetime.datetime(2010, 6, 1, tzinfo=dateutil.tz.tzutc())
today =datetime.date.today()
to_date = datetime.datetime(today.year, today.month, today.day, tzinfo=dateutil.tz.tzutc())


class IssuetrackerAPI(_ISAPI):
    
    def make_pickle_cache(self, *args):
        issues = super().download_issues(*args)
        with open("gantt_dummy.pkl", 'wb') as f:
            pickle.dump(issues, f)
        return issues
    def download_issues(self, *args):
        if not os.path.exists("gantt_dummy.pkl"):
            return self.make_pickle_cache(*args)
        else:
            with open("gantt_dummy.pkl", 'rb') as f:
                try:
                    issues = pickle.load(f)
                except (pickle.PickleError, OSError, EOFError):
                    pass
                else:
                    return issues
            return self.make_pickle_cache(*args)
        return issues


# 8/15/17

In [2]:
from scripts.tools.issuetracker.frs_update import main
import shutil
import datetime

def bt():
    return datetime.date.today().strftime("%y%m%d")
    
pdfpth = "Software_FRS_" + bt()

try:
    shutil.rmtree(pdfpth)
except FileNotFoundError:
    pass
print("Begin Main", flush=True)
res = main("issue.pbsbiotech.com", 'nstarkweather', 'kookychemist', 'pbssoftware', None, 
     "today", 'Hello v3.0 FRS Update', pdfpth, "FRS Update", 
     "IS00038D2 Rev A Functional Requirements Specification")
#os.startfile('"%s"'%res)
print()
print("Done")

Begin Main

Logging into Issuetracker...Downloading projects...

Got update: PGS_BEGIN_PROCESS_PDFS 
Creating PDF Index...
Creating Word Document...
Creating index 14 of 92                    
Creating index 40 of 92                    
Creating index 65 of 92                    
Creating index 92 of 92                    
Got update: PGS_IPG_FLAG_WAITING 
Empty Queue, exiting download worker #0
Empty Queue, exiting download worker #1
Empty Queue, exiting download worker #3
Empty Queue, exiting download worker #7
Empty Queue, exiting download worker #6
Empty Queue, exiting download worker #5
Empty Queue, exiting download worker #4
Empty Queue, exiting download worker #2
Got update: MPD_ALL_COMPLETE 
Got update: PGS_PDFTK_INFO_BEGIN 
Getting PDF Page Info (92/92)                    
Got update: PGS_PDFTK_INFO_DONE 
Got update: PGS_PROCESS_PDFS_DONE 
Got update: PGS_PREPARE_POSTSCRIPT 
Got update: PGS_BEGIN_BIND_PDF 
Got update: PGS_DONE 
Done


In [1]:
#  5/28/2020

import os
os.chdir(os.path.join(os.path.abspath(os.path.expanduser("~")), "Documents", "Personal", "Test"))

from scripts.tools.issuetracker.frs_update import main
import shutil
import datetime

def bt():
    return datetime.date.today().strftime("%y%m%d")
    
pdfpth = "Software_FRS_" + bt()

try:
    shutil.rmtree(pdfpth)
except FileNotFoundError:
    pass
print("Begin Main", flush=True)
res = main("issue.pbsbiotech.com", 'nstarkweather', 'kookychemist', 'pbssoftware', None, 
     "today", 'Hello v3.0 FRS Update', pdfpth, "FRS Update", 
     "IS00038D2 Rev A Functional Requirements Specification")
#os.startfile('"%s"'%res)
print()
print("Done")

Begin Main

Logging into Issuetracker...Downloading projects...


AttributeError: 'itertools.chain' object has no attribute 'values'